In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-generated-essay-using-palm-from-google-gen-ai/LLM_generated_essay_PaLM.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/combined/train_essay_combined.csv


In [2]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, SimpleRNN, LSTM , GRU, Bidirectional, Embedding
from keras.layers import Dropout
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
train = pd.read_csv("/kaggle/input/combined/train_essay_combined.csv")

In [4]:
train["text"].astype(str)

0       Cars. Cars have been around since they became ...
1       Transportation is a large necessity in most co...
2       "America's love affair with it's vehicles seem...
3       How often do you ride in a car? Do you drive a...
4       Cars are a wonderful thing. They are perhaps o...
                              ...                        
2757    Dear Senator,\n\nI am writing to you today to ...
2758    Dear Senator,\n\nI am writing to you today to ...
2759    Dear Senator,\n\nI am writing to you today to ...
2760    Dear Senator,\n\nI am writing to you today to ...
2761    Dear Senator,\n\nI am writing to you today to ...
Name: text, Length: 2762, dtype: object

In [5]:
train_split, val_split = train_test_split(train, test_size=0.4, shuffle=True)

In [6]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.tokenizer
train_essays = []
for _, row in train_split.iterrows():
  train_essays.append([token.text for token in tokenizer(row["text"])])

val_essays = []
for _,row in val_split.iterrows():
  val_essays.append([token.text for token in tokenizer(row["text"])])


In [7]:
all_essays = train_essays+val_essays
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(all_essays)]
doc2vec_model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

In [8]:
train_vectors = np.array([doc2vec_model.infer_vector(essay) for essay in train_essays])
train_labels = np.array([x["generated"] for _, x in train_split.iterrows()])
val_vectors = np.array([doc2vec_model.infer_vector(essay) for essay in val_essays])
val_labels = np.array([x["generated"] for _, x in val_split.iterrows()])

In [9]:
nn = Sequential()
nn.add(Dense(64, activation="relu"))
nn.add(Dense(32, activation="relu"))
nn.add(Dense(1, activation="sigmoid"))

nn.compile(loss=keras.losses.BinaryCrossentropy(), optimizer="adam", metrics=["accuracy"])
nn.fit(x=train_vectors, y=train_labels, epochs=10, validation_data=(val_vectors, val_labels),
       batch_size=2,callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)] )

Epoch 1/10
829/829 [==============================] - 4s 3ms/step - loss: 0.0561 - accuracy: 0.9837 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 2/10
829/829 [==============================] - 2s 3ms/step - loss: 0.0048 - accuracy: 0.9994 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 3/10
829/829 [==============================] - 2s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 7.4553e-04 - val_accuracy: 1.0000
Epoch 4/10
829/829 [==============================] - 3s 3ms/step - loss: 2.3570e-04 - accuracy: 1.0000 - val_loss: 4.1434e-04 - val_accuracy: 1.0000
Epoch 5/10
829/829 [==============================] - 2s 3ms/step - loss: 7.9651e-05 - accuracy: 1.0000 - val_loss: 3.6992e-04 - val_accuracy: 1.0000
Epoch 6/10
829/829 [==============================] - 2s 3ms/step - loss: 4.1582e-05 - accuracy: 1.0000 - val_loss: 2.7559e-04 - val_accuracy: 1.0000
Epoch 7/10
829/829 [==============================] - 3s 3ms/step - loss: 2.4337e-05 - accuracy: 1.0000 - val_loss: 2.02

In [10]:
test_split = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
test_essays = []
for _,row in test_split.iterrows():
  test_essays.append([token.text for token in tokenizer(row["text"])])

In [11]:
from sklearn.metrics import accuracy_score
predictions = nn.predict(np.array([doc2vec_model.infer_vector(essay) for essay in test_essays]))
submission = pd.DataFrame()
submission["id"] = test_split["id"]
submission["generated"] = predictions
submission.to_csv("/kaggle/working/submission.csv", index=False)

1/1 [==============================] - 0s 79ms/step
